In [ ]:
experiment = 'kialov2-domains-no_hard_negatives-max_candidates_500'
# experiment = 'one_map-batch_32'
# experiment = 'one_map-batch_8'

metrics = ['mrr', 'p1', 'p5']
metrics = ['mrr', 'p1', 'p5', 'dist']
# metrics = ['p1']

variant = 'all'
variant = 'only_leafs'
variant = 'only_leafs_limited_types'

ext = 'svg'
ext = 'pdf'

In [ ]:
from kialo_domains_util import get_subtopic_to_parent
sub2maintopic, main2subtopic = get_subtopic_to_parent('data/kialo_domains.tsv')
domains = list(main2subtopic.keys())
domains

In [ ]:
import pandas as pd
from pathlib import Path
import json

data_path = Path.home() / 'data/delib-results/results'

def read_results(path):
    print(path)
#     print(list(data_path.glob(path)))
    all_results = []
    for i, f in enumerate(data_path.glob(path)):
        if not str(f).endswith('-train.json'):
            results = json.loads(f.read_text())
#             print(f)
#             print(results)
#             all_results.append({'source': s[s.find('trained-on-')+len('trained-on-'):s.rfind('-')] if 'trained-on-' in s else 'source', 'target': f.stem, **{metric: results[variant][metric] for metric in metrics}})
            all_results.append({'source': domains[int(f.parent.parent.name.split('-')[0][len('domain'):])], 'target': f.parent.name, **{metric: results[variant][metric] for metric in metrics}})

    df = pd.DataFrame(all_results)
    return df
df = read_results(f'{experiment}/*-results/*/-avg.json')
df

In [ ]:
df[df['source']=='religion']

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.pyplot import cm
from matplotlib.patches import Rectangle
import numpy as np

def plot(metric, ax):
    data = df.pivot_table(values=metric, index='source', columns='target')
#     display(data)
    mask = np.zeros_like(data, dtype=np.bool)
#     mask[np.triu_indices_from(mask)] = remove_triangle
#     mask[np.eye(mask.shape[0], dtype=bool)] = 1
    
    g = sns.heatmap(data, annot=True, 
#                     cmap=sns.cubehelix_palette(start=.5, rot=-.5, dark=0.3, light=.80, reverse=True, as_cmap=True),
                    cmap=cm.Blues,
                    cbar=False, ax=ax,
                    fmt='.3f',
                   mask=mask)
    
    data[np.eye(mask.shape[0], dtype=bool)] = 0

#     row_max = data.idxmax(axis=1)
#     for row, index in enumerate(data.columns):
#         position = data.columns.get_loc(row_max[index])
#         ax.add_patch(Rectangle((position, row),1,1, fill=False, edgecolor='red', lw=3))
        
    row_max = data.idxmax(axis=0)
#     display(row_max)
    for col, index in enumerate(data.columns):
        position = data.index.get_loc(row_max[index])
        ax.add_patch(Rectangle((col, col),1,1, fill=False, edgecolor='grey', lw=3))
        ax.add_patch(Rectangle((col, position),1,1, fill=False, edgecolor='red', lw=3))
        
    ax.set_title(metric)
    g.set_xticklabels(g.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor');
    fig = g.get_figure()
    plt.tight_layout()
    return fig

In [ ]:
f, axes = plt.subplots(len(metrics), 1, figsize=(10, 8*len(metrics)))

for metric, ax in zip(metrics, axes):
    fig=plot(metric, ax)

# fig.savefig(f'results/kialo-{variant}-{"-".join(metrics)}.{ext}', format=ext)

In [ ]:
df_same = df[df['source']==df['target']]
df_same['source'] = 'same'
df = df_same

In [ ]:
f, axes = plt.subplots(1, len(metrics), figsize=(12*len(metrics), 1))

for i, metric in enumerate(metrics):
    fig=plot(metric, i)

# fig.savefig(f'results/{variant}-{"-".join(metrics)}.{ext}', format=ext)

In [ ]:
# df = read_results('xlm-roberta-base-evaluated-on-*-results/*')
# df = read_results('xlm-roberta-base-dev-evaluated-on-*-dev-*-results/*')
df = read_results(f'{experiment}/*-results/*')
df['source'] = 'others'
df

In [ ]:
f, axes = plt.subplots(1, len(metrics), figsize=(4*len(metrics), 0.5))

for i, metric in enumerate(metrics):
    fig=plot(metric, i)

fig.savefig(f'results/others-{experiment}-{variant}-{"-".join(metrics)}.{ext}', format=ext)